In [24]:
import osmnx as ox
import networkx as nx
import pandas as pd
import json
import requests
import random

from rtree import index
from shapely.geometry import box
from utils import *
from collections import defaultdict

In [2]:
G = ox.save_load.load_graphml(filename="Singapore_drive.graphml")
G_ped = ox.save_load.load_graphml(filename="Singapore_walk.graphml")
nodes_central = pd.read_pickle('data/Central_nodes.pkl')
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

print(type(G))

<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [3]:
# Calculate congestion of network for a time slice

f = open("../../Traffic speed bands/Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)
speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)
calculate_congestion(G, edges, speed_bands)

edges.head()

/home/orionstar25/Semester-8-Major-Project/Open Street Maps/Phase 5 - System Optimal Social Model/utils.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MaximumSpeed'][i] = '80'


,u,v,key,osmid,oneway,name,highway,maxspeed,length,travel_time,Location,geometry,observed_speed,capacity
0,1820000257,1865254946,0,174765824,True,Tampines Avenue 8,tertiary,60.0,14.248,0.85488,"(1.3565805, 103.9326163, 1.3566943, 103.9326752)","LINESTRING (103.93262 1.35658, 103.93268 1.35669)",31.5,4
1,1820000257,4662056865,0,587044201,True,Tampines Avenue 5,primary,60.0,27.282,1.63692,"(1.3565805, 103.9323996, 1.3566955, 103.9326163)","LINESTRING (103.93262 1.35658, 103.93240 1.35670)",31.5,8
2,3874553858,6995781882,0,384254693,False,Fidelio Street,tertiary,40.0,20.740,1.8666,"(1.3191823, 103.9212368, 1.3192215, 103.9214192)","LINESTRING (103.92124 1.31918, 103.92142 1.31922)",34,9
3,3874553858,3874553861,0,384254631,True,Fidelio Street,tertiary,40.0,15.191,1.3671900000000001,"(1.3191111, 103.9211207, 1.3191823, 103.9212368)","LINESTRING (103.92124 1.31918, 103.92112 1.31911)",34,7
4,1820000259,245190289,0,170861145,True,Tampines Avenue 8,tertiary,60.0,13.573,0.8143800000000001,"(1.3565404, 103.9326858, 1.3566471, 103.9327451)","LINESTRING (103.93275 1.35665, 103.93269 1.35654)",31.5,4


In [4]:
# Filter out only central SG nodes for trip requests
# key: node label ,value: location
idx = index.Index()

xx = nodes['x'].astype(float)
yy = nodes['y'].astype(float)
node_id = nodes['osmid']

for node, x, y in zip(node_id, xx, yy):
    idx.insert(int(node), (x,y,x,y))

In [ ]:
# NEED TO CHANGE INTO NEW SG.OSM FOR DIFFERENT TIME SLICE (BECAUSE OF BPR VALUES AS WEIGHT)

### Middle mile routing

In [5]:
def modified_hybrid_search(G, x):
    n = len(x)
    cutoff = math.floor(n/math.exp(1))
    min_pair = x[0]
    source = min_pair[0][0]
    dest = min_pair[0][1]
    min_tt = float('inf')

    for i in range(n):
        source_x = nodes[nodes['osmid'] == source]['x'].values[0]
        source_y = nodes[nodes['osmid'] == source]['y'].values[0]
        dest_x = nodes[nodes['osmid'] == dest]['x'].values[0]
        dest_y = nodes[nodes['osmid'] == dest]['y'].values[0]

        url = "http://0.0.0.0:5000/route/v1/driving/{},{};{},{}".format(source_x, source_y, dest_x, dest_y)
        r = requests.get(url)
        json = r.json()            
        travel_time = json['routes'][0]['weight']

        # have reached cutoff and didnt find optimal yet
        if i > cutoff:
            if travel_time < min_tt:
                min_tt = travel_time
                min_pair = x[i]
                break
            else:
                break
        else:
            if travel_time < min_tt:
                min_tt = travel_time
                min_pair = x[i]

                if i < n-1:
                    if travel_time < x[i+1][1]:
                        break
                    else:
                        source = x[i+1][0][0]
                        dest = x[i+1][0][1]
                else:
                    break
            else:
                if i < n-1:
                    if min_tt < x[i+1][1]:
                        break
                    else:
                        source = x[i+1][0][0]
                        dest = x[i+1][0][1]
                else:
                    break

    return min_tt, min_pair[0]

In [7]:
# get 10 random sources from central SG
sources = []
for i in range(4):
    r = random.randint(0,5640)
    s = nodes_central['osmid'].values[r]
    sources.append(s)

# get 20 corresponding random targets from central SG
targets = []
for i in range(5):
    r = random.randint(0,5640)
    t = nodes_central['osmid'].values[r]
    targets.append(t)

# Find optimal transit points for the middle mile
AD = []
demand = []
all_miles = dict()

for s in sources:
    for t in targets:
        print(s, t)
        AD.append([s,t])
        
        orig_point = nodes[nodes['osmid'] == s].geometry.values[0]
        target_point = nodes[nodes['osmid'] == t].geometry.values[0]
        orig_xy = (orig_point.y, orig_point.x)
        target_xy = (target_point.y, target_point.x)

        # Max distance for walking is 720m (10 min * 1.2 m/s)
        # Find all nodes within 720 m from candidate sources and candidate destinations
        candidate_dest = find_nearest_nodes(nodes, target_xy, 400, idx)
        candidate_source = find_nearest_nodes(nodes, orig_xy, 400, idx)

        if len(candidate_dest) == 0:
            candidate_dest.append(ox.get_nearest_node(G, target_xy))

        if len(candidate_source) == 0:
            candidate_source.append(ox.get_nearest_node(G, orig_xy))

        euclidean = get_length_dict(nodes, candidate_source, candidate_dest)
        _, OD = modified_hybrid_search(G, euclidean)
        demand.append(OD)
        
        # first mile
        all_miles[()]
        # last mile
    
trips = dict()
for d in demand:
    trips[d[0]] = list()
    
for d in demand:
    trips[d[0]].append(d[1])
trips

4715586520 378006753
4715586520 309872846
4715586520 388122028
4715586520 3371653868
4715586520 4593195725
5992286923 378006753
5992286923 309872846
5992286923 388122028
5992286923 3371653868
5992286923 4593195725
7030663130 378006753
7030663130 309872846
7030663130 388122028
7030663130 3371653868
7030663130 4593195725
5239943342 378006753
5239943342 309872846
5239943342 388122028
5239943342 3371653868
5239943342 4593195725


In [57]:
# Convert to .tntp files for Frank Wolfe
# make_nodes_tntp(nodes)
# make_net_tntp(edges)
# make_trips_tntp(trips)

No. of requests:  40


## First mile

In [12]:
# use pedestrain network only----> change this
def print_route(route):
  i = 0
  n = len(route)

  for i in range(n):
    for nei,w in G_whole[route[i]].items():
      if i+1 != n:
        if nei == route[i+1]:
          print(w[0].get('highway'), w[0].get('length'))

In [31]:
orig_point = nodes[nodes['osmid'] == AD[0][0]].geometry.values[0]
orig_xy = (orig_point.y, orig_point.x)
orig_node, dist = ox.get_nearest_node(G_ped, orig_xy, method='haversine', return_dist = True)

target_point = nodes[nodes['osmid'] == demand[0][0]].geometry.values[0]
target_xy = (target_point.y, target_point.x)
target_node = demand[0][0]

first_mile_length, first_mile = nx.single_source_dijkstra(G_ped, orig_node, target_node, weight='length')

print_route(first_mile)
print("Distance from X to A: ", dist) # in metres
print("Distance from A to B: ", first_mile_length) # in metres

residential 102.051
residential 7.747
residential 32.174
residential 124.57300000000001
residential 56.33
Distance from X to A:  482.23028553412615
Distance from A to B:  322.875


### Last Mile

In [32]:
target_node, dist = ox.get_nearest_node(G_whole, target_xy, method='haversine', return_dist= True)
last_mile = nx.shortest_path(G_whole, sd_pair[0][1], target_node, weight='length', method='dijkstra')
last_mile_length = nx.shortest_path_length(G_whole, sd_pair[0][1], target_node, weight='length', method='dijkstra')

print_route(last_mile)
print("Distance from C to D: ", last_mile_length)
print("Distance from D to Y: ", dist)

primary 561.0409999999999
primary 31.486
primary 10.98
primary 17.553
primary 11.666
primary 26.612
primary 256.842
primary 9.377
tertiary 13.573
primary 8.92
Distance from C to D:  948.0499999999998
Distance from D to Y:  0.0
